In [2]:
#Connect Ollama to LangChain

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

INDEX_DIR = "data/index/faiss_merck_hf"  # change if yours differs
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
TOP_K = 5

embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
vectorstore = FAISS.load_local(INDEX_DIR, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Create the Ollama LLM
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0
)

In [4]:
#Build a grounded prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You answer using ONLY the provided context from a medical manual. "
     "If the answer is not in the context, say: 'I don't know based on the provided document.' "
     "Cite sources as (page=<page>, chunk_id=<chunk_id>) after each key claim."),
    ("human", "Question: {question}\n\nContext:\n{context}\n\nAnswer:")
])



In [5]:
#RAG functions

def format_context(docs):
    parts = []
    for d in docs:
        page = d.metadata.get("page", "?")
        chunk_id = d.metadata.get("chunk_id", "?")
        parts.append(f"(page={page}, chunk_id={chunk_id})\n{d.page_content}")
    return "\n\n---\n\n".join(parts)

def rag_answer(question: str, k: int = TOP_K):
    docs = retriever.invoke(question)  # new LangChain API
    context = format_context(docs[:k])
    msg = prompt.invoke({"question": question, "context": context})
    resp = llm.invoke(msg)
    return resp.content, docs[:k]


In [6]:
# Test

question = "What is the approach to a patient with anemia?"
answer, sources = rag_answer(question)

print("ANSWER:\n", answer)
print("\nSOURCES (page, chunk_id):")
for d in sources:
    print(d.metadata.get("page", "?"), d.metadata.get("chunk_id", "?"))


ANSWER:
 The approach to a patient with anemia involves:

1. Evaluating for acute or chronic blood loss through history, examination, and stool tests (page=1056, chunk_id=4399).
2. If blood loss is not detected, laboratory testing is done to determine if the anemia is due to deficient RBC production or excessive hemolysis (page=1056, chunk_id=4399).
3. Considering risk factors for particular anemias and symptoms that reflect the underlying disorder through history (page=1056, chunk_id=4399).
4. Performing laboratory tests such as serum folate and B12 levels for macrocytic anemias (page=2802, chunk_id=11923).
5. Consulting a hematologist if initial tests are nondiagnostic or there is no response to empiric treatment (page=2802, chunk_id=11923).

Treatment depends on the cause of the anemia and may involve:

1. Transfusion for severe constitutional symptoms or cardiopulmonary symptoms (page=1057, chunk_id=4408).
2. Eliminating or reducing drugs that cause megaloblastic states (page=1069,